In [1]:
# %pip install optuna

In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

d:\Programming\Learn_Pytorch_for_deep_learning\.env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [6]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-09-17 09:57:29,709] A new study created in memory with name: no-name-3d565b69-7b97-4c90-bb38-27d00ee78437
[I 2025-09-17 09:57:29,870] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 58, 'max_depth': 20}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-17 09:57:30,296] Trial 1 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 185, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-17 09:57:30,594] Trial 2 finished with value: 0.7765363128491621 and parameters: {'n_estimators': 118, 'max_depth': 20}. Best is trial 2 with value: 0.7765363128491621.
[I 2025-09-17 09:57:30,912] Trial 3 finished with value: 0.7802607076350093 and parameters: {'n_estimators': 119, 'max_depth': 18}. Best is trial 3 with value: 0.7802607076350093.
[I 2025-09-17 09:57:31,409] Trial 4 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 189, 'max_depth': 14}. Best is trial 3 with value: 0.780260

In [7]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 119, 'max_depth': 18}


In [8]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


## Samplers in Optuna

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [10]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-09-17 09:57:46,989] A new study created in memory with name: no-name-56b2e43e-5f89-492a-9de7-685c526e5eaa
[I 2025-09-17 09:57:47,550] Trial 0 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 175, 'max_depth': 11}. Best is trial 0 with value: 0.7672253258845437.
[I 2025-09-17 09:57:47,718] Trial 1 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 61, 'max_depth': 6}. Best is trial 0 with value: 0.7672253258845437.
[I 2025-09-17 09:57:48,230] Trial 2 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 166, 'max_depth': 9}. Best is trial 0 with value: 0.7672253258845437.
[I 2025-09-17 09:57:48,382] Trial 3 finished with value: 0.7579143389199254 and parameters: {'n_estimators': 61, 'max_depth': 4}. Best is trial 0 with value: 0.7672253258845437.
[I 2025-09-17 09:57:48,742] Trial 4 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 131, 'max_depth': 11}. Best is trial 0 with value: 0.767225325

In [11]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7765363128491621
Best hyperparameters: {'n_estimators': 114, 'max_depth': 20}


In [12]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


In [13]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [14]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-09-17 09:58:03,131] A new study created in memory with name: no-name-84618e92-cf97-4754-82cc-c33cee57d99e
[I 2025-09-17 09:58:03,375] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-17 09:58:03,764] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-17 09:58:03,901] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-09-17 09:58:04,173] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-09-17 09:58:04,459] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [15]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [16]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Optuna Visualizations

In [17]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [18]:
# 1. Optimization History
plot_optimization_history(study).show()

In [19]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [20]:
# 3. Slice Plot
plot_slice(study).show()

In [21]:
# 4. Contour Plot
plot_contour(study).show()

In [22]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models

In [23]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [24]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [25]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-09-17 09:58:10,788] A new study created in memory with name: no-name-8a44e4dc-3e4f-4151-bb4a-63378721e7dd
[I 2025-09-17 09:58:10,935] Trial 0 finished with value: 0.7821229050279329 and parameters: {'classifier': 'RandomForest', 'n_estimators': 66, 'max_depth': 18, 'min_samples_split': 9, 'min_samples_leaf': 9, 'bootstrap': False}. Best is trial 0 with value: 0.7821229050279329.
[I 2025-09-17 09:58:10,950] Trial 1 finished with value: 0.702048417132216 and parameters: {'classifier': 'SVM', 'C': 12.826412911406658, 'kernel': 'sigmoid', 'gamma': 'auto'}. Best is trial 0 with value: 0.7821229050279329.
[I 2025-09-17 09:58:11,344] Trial 2 finished with value: 0.7411545623836128 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 80, 'learning_rate': 0.16643003608089052, 'max_depth': 8, 'min_samples_split': 9, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.7821229050279329.
[I 2025-09-17 09:58:11,354] Trial 3 finished with value: 0.7430167597765364 and paramet

In [26]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.12045722121878354, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7895716945996275


In [27]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.782123,2025-09-17 09:58:10.789060,2025-09-17 09:58:10.935418,0 days 00:00:00.146358,NaN,False,RandomForest,NaN,NaN,NaN,18.0,9.0,9.0,66.0,COMPLETE
1,1,0.702048,2025-09-17 09:58:10.935418,2025-09-17 09:58:10.950260,0 days 00:00:00.014842,12.826413,NaN,SVM,auto,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
2,2,0.741155,2025-09-17 09:58:10.952043,2025-09-17 09:58:11.343980,0 days 00:00:00.391937,NaN,NaN,GradientBoosting,NaN,NaN,0.166430,8.0,10.0,9.0,80.0,COMPLETE
3,3,0.743017,2025-09-17 09:58:11.344568,2025-09-17 09:58:11.354605,0 days 00:00:00.010037,0.497459,NaN,SVM,auto,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
4,4,0.744879,2025-09-17 09:58:11.354605,2025-09-17 09:58:12.095458,0 days 00:00:00.740853,NaN,NaN,GradientBoosting,NaN,NaN,0.012078,9.0,2.0,7.0,91.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.785847,2025-09-17 09:58:27.224592,2025-09-17 09:58:27.254335,0 days 00:00:00.029743,8.139043,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.735568,2025-09-17 09:58:27.257284,2025-09-17 09:58:29.045171,0 days 00:00:01.787887,NaN,NaN,GradientBoosting,NaN,NaN,0.025380,19.0,4.0,3.0,190.0,COMPLETE
97,97,0.709497,2025-09-17 09:58:29.045171,2025-09-17 09:58:29.057396,0 days 00:00:00.012225,0.116673,NaN,SVM,auto,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.787709,2025-09-17 09:58:29.057396,2025-09-17 09:58:29.071207,0 days 00:00:00.013811,0.156120,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [28]:
study.trials_dataframe()['params_classifier'].value_counts()

params_classifier
SVM                 71
RandomForest        19
GradientBoosting    10
Name: count, dtype: int64

In [29]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

params_classifier
GradientBoosting    0.750279
RandomForest        0.768892
SVM                 0.774412
Name: value, dtype: float64

In [30]:
# 1. Optimization History
plot_optimization_history(study).show()

In [31]:
# 3. Slice Plot
plot_slice(study).show()

In [32]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [33]:
# %pip install xgboost

In [34]:
%pip install optuna-integration[xgboost]

Note: you may need to restart the kernel to use updated packages.


In [35]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2025-09-17 09:58:30,420] A new study created in memory with name: no-name-b92769c1-d07f-41ee-9f92-6097146fd427


[0]	train-mlogloss:0.78414	eval-mlogloss:0.77606
[1]	train-mlogloss:0.58910	eval-mlogloss:0.56575
[2]	train-mlogloss:0.45816	eval-mlogloss:0.42436
[3]	train-mlogloss:0.36308	eval-mlogloss:0.32308
[4]	train-mlogloss:0.30313	eval-mlogloss:0.25701
[5]	train-mlogloss:0.25761	eval-mlogloss:0.20292
[6]	train-mlogloss:0.23097	eval-mlogloss:0.17255
[7]	train-mlogloss:0.21890	eval-mlogloss:0.15901
[8]	train-mlogloss:0.19965	eval-mlogloss:0.13436
[9]	train-mlogloss:0.19357	eval-mlogloss:0.12464
[10]	train-mlogloss:0.18801	eval-mlogloss:0.11884
[11]	train-mlogloss:0.18625	eval-mlogloss:0.11615
[12]	train-mlogloss:0.18582	eval-mlogloss:0.11572
[13]	train-mlogloss:0.18536	eval-mlogloss:0.11514
[14]	train-mlogloss:0.18527	eval-mlogloss:0.11480
[15]	train-mlogloss:0.18521	eval-mlogloss:0.11467
[16]	train-mlogloss:0.18453	eval-mlogloss:0.11424
[17]	train-mlogloss:0.18347	eval-mlogloss:0.11382
[18]	train-mlogloss:0.18332	eval-mlogloss:0.11456
[19]	train-mlogloss:0.18241	eval-mlogloss:0.11415
[20]	train

[I 2025-09-17 09:58:30,902] Trial 0 finished with value: 1.0 and parameters: {'lambda': 4.057323911367664e-07, 'alpha': 0.15591987079988331, 'eta': 0.27167587789575615, 'gamma': 6.789690775557702e-05, 'max_depth': 4, 'min_child_weight': 7, 'subsample': 0.7827374274731098, 'colsample_bytree': 0.9794792471971193}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:0.82168	eval-mlogloss:0.80540
[1]	train-mlogloss:0.72853	eval-mlogloss:0.70082
[2]	train-mlogloss:0.59228	eval-mlogloss:0.55228
[3]	train-mlogloss:0.47172	eval-mlogloss:0.42300
[4]	train-mlogloss:0.39696	eval-mlogloss:0.34391
[5]	train-mlogloss:0.33991	eval-mlogloss:0.28379
[6]	train-mlogloss:0.30463	eval-mlogloss:0.24546
[7]	train-mlogloss:0.30467	eval-mlogloss:0.24530
[8]	train-mlogloss:0.27576	eval-mlogloss:0.21067
[9]	train-mlogloss:0.26558	eval-mlogloss:0.19520
[10]	train-mlogloss:0.25905	eval-mlogloss:0.18571
[11]	train-mlogloss:0.25905	eval-mlogloss:0.18571
[12]	train-mlogloss:0.25892	eval-mlogloss:0.18587
[13]	train-mlogloss:0.25893	eval-mlogloss:0.18608
[14]	train-mlogloss:0.25895	eval-mlogloss:0.18586
[15]	train-mlogloss:0.25893	eval-mlogloss:0.18595
[16]	train-mlogloss:0.25892	eval-mlogloss:0.18595
[17]	train-mlogloss:0.25889	eval-mlogloss:0.18596
[18]	train-mlogloss:0.25904	eval-mlogloss:0.18572
[19]	train-mlogloss:0.25896	eval-mlogloss:0.18582
[20]	train

[I 2025-09-17 09:58:31,056] Trial 1 finished with value: 1.0 and parameters: {'lambda': 0.03650007112184016, 'alpha': 0.0028278533066872892, 'eta': 0.2382266641299075, 'gamma': 0.7509705991067698, 'max_depth': 4, 'min_child_weight': 8, 'subsample': 0.6521413022527576, 'colsample_bytree': 0.5632251280097232}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:0.92374	eval-mlogloss:0.91875
[1]	train-mlogloss:0.81136	eval-mlogloss:0.79658


[I 2025-09-17 09:58:33,643] Trial 48 pruned. Trial was pruned at iteration 2.


[0]	train-mlogloss:1.00495	eval-mlogloss:1.00173
[1]	train-mlogloss:0.96346	eval-mlogloss:0.95433


[I 2025-09-17 09:58:33,669] Trial 49 pruned. Trial was pruned at iteration 2.


Best trial: {'lambda': 4.057323911367664e-07, 'alpha': 0.15591987079988331, 'eta': 0.27167587789575615, 'gamma': 6.789690775557702e-05, 'max_depth': 4, 'min_child_weight': 7, 'subsample': 0.7827374274731098, 'colsample_bytree': 0.9794792471971193}
Best accuracy: 1.0


In [36]:
# ! pip install optuna-integration[xgboost]

In [37]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()